# Parte extra de la tarea: escribir digitos para MNIST

Tarea Deep Learning.

In [3]:
import torchvision
from torch.utils.data import DataLoader

## Escribir los dígitos personalizados para MNIST

Siga las instrucciones para generar dígitos manuscritos en el mismo formato del dataset MNIST.
El siguiente script inicia un recuadro donde puede dibujar su dígito, indicando a qué número pertenece.

In [ ]:
!pip install Js2Py

In [5]:
from IPython.display import HTML, Image
#from google.colab.output import eval_js
from js2py import eval_js
from base64 import b64decode
import string
import random
import os

letters = string.ascii_lowercase

canvas_html = """
<canvas width=%d height=%d></canvas>
<button>Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.fillStyle = 'black'
ctx.fillRect(0, 0, canvas.width, canvas.height)
ctx.lineWidth = %d
ctx.strokeStyle = 'white'
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}
canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/jpeg'))
  }
})
</script>
"""

def draw(label, w=280, h=280, line_width=10):
    display(HTML(canvas_html % (w, h, line_width)))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])
    path_label = f"user_numbers/{label}/"
    file_name = ''.join(random.choice(letters) for i in range(10)) + ".jpg"
    os.makedirs(path_label, exist_ok=True)
    with open(path_label + file_name, 'wb') as f:
        f.write(binary)
    return binary

Para dibujar tiene que hacer una llamada a la función `draw` indicando el número que quiere dibujar. El script creará la carpeta `user_numbers`, donde se irán generando carpetas divididas con los labels de cada número. Por ejemplo, si decide dibujar el `1`, se creará la carpeta `user_numbers/1/` con una imagen dentro que corresponderá al dígito que acaba de dibujar. Puede ver esto en el panel del lado izquierdo, en la pestaña `Files`.

In [6]:
draw(label=1)

JsException: ReferenceError: data is not defined



---



## Leer las imágenes generadas

Para cargar las imagenes que generó debe hacer uso de la conveniente clase de Pytorch `ImageFolder` y luego de `DataLoader`.
Para esto utilize el siguiente script.

In [ ]:
user_made = torchvision.datasets.ImageFolder(
        root="user_numbers/",
        transform=torchvision.transforms.Compose([
            # transformamos las imagenes a blanco y negro
            torchvision.transforms.Grayscale(num_output_channels=1),
            # achicamos la imagen de 280x280 a 28x28 (mas facil dibujar en 280)
            torchvision.transforms.Resize((28, 28), interpolation=2),
            # a tensor de pytorch
            torchvision.transforms.ToTensor()
        ])
    )
# Gracias por las limitaciones pytorch...
user_made.target_transform = lambda _id: int(list(user_made.class_to_idx.keys())[list(user_made.class_to_idx.values()).index(_id)])
dataloader_user = DataLoader(user_made, batch_size=1)